In [ ]:
import sys
sys.path.insert(0,'../scripts')
from pycap_for_PESTPP_MOU import postprocess_MOU, plot_pareto, prep_for_viz, create_viz_app, plot_pareto_with_scenarios
from pathlib import Path
from ipywidgets import interact, fixed
import yaml



# Read in one of the configuration files and postprocess it. 
### The `config_file` should be your `secnario_name` + `".yml"` as assigned in the `05_MOU_setup_run.ipynb` notebook. 

In [ ]:
config_file = 'fish_cash_baseline.yml'

with open(Path('./configurations') / config_file,'r') as ifp:
    inpars = yaml.safe_load(ifp)
inpars

### Below, the `config_file` is processed, beginning the postprocessing

In [ ]:
run_path = Path(inpars['run_path'])
run_name = inpars['scenario_name']
pareto_df = postprocess_MOU(run_name, run_path)

In [ ]:
pareto_df

### Next the evolution of the pareto frontier is plotted, over MOU iterations. Check out how the algorithm converges on a well-defined pareto tradeoff frontier

In [ ]:
interact(plot_pareto,  currgen=(0,50,1), pareto_df=fixed(pareto_df));

### Where do your scenarios line up? You can provide a list of scenarios that you ran in notebook #3 in the following function to plot a red "x" where each scenario plots up

### NOTE: This only works for the fish vs. receipts case!

In [ ]:
dollars=False
if 'dollar' in inpars['objectives']:
    dollars = True
if dollars:
    plot_pareto_with_scenarios(pareto_df, scenarios=['tryone','tryanother'])

### Finally, there is an interactive plot with the pareto frontier on the left and a map of corresponding "optimal" pumping rates on the right. 

### Note - if you get an error about ports being in conflict, change the `port=4242` argument to a different number than `4242`

In [ ]:

pareto_df_final, dv_df =  prep_for_viz(pareto_df, 50, run_path, run_name, dollar_objective=dollars)
app = create_viz_app(pareto_df_final, dv_df)
app.run(port=4242)

### check out `pareto_df_final` and `dv_df`. What's in these datasets? What further analysis can you perform with this information?

In [ ]:
dv_df.sample(10)

In [ ]:
# If we want to, we can save out the pareto tradeoff objectives results to a file. 
# Just change the name below and it will be a new copy!
pareto_df_final[['Depletion (cfs)','Total Pumping (cfs)']].to_csv('tradeoff.csv')

In [ ]:
pareto_df.columns